# Project Proposal (think of a better title)

**Group:** 3

## Introduction

### Background information

### Dataset

## Exploratory Data Analysis

## Methods

## Expected Outcomes and Significance

### Expected outcomes

### Significance

### Future questions